In [1]:
!cat .env

STAGE=PRODUCTION


In [2]:
import json
from tqdm.notebook import tqdm
from openpyxl import load_workbook
from django.contrib.gis.geos import Point
from helpers.signals import DisableSignals
from helpers.strings import get_autonumber_array
from simple_history.utils import bulk_create_with_history
from uuid import uuid4
from django.utils import timezone
import random
from datetime import timedelta
from datetime import datetime
from collections import defaultdict

In [3]:
arteriscs = User.objects.get(username="rlcs")

In [4]:
company = Company.objects.get(name="Homolog - CCR")

In [5]:
def get_sheet_values(filename, sheet_index):
    wb = load_workbook(filename + ".xlsx")
    sheetname = wb.sheetnames[sheet_index]
    ws = wb[sheetname]
    
    header = []
    values = []

    for index, row in enumerate(ws.rows):
        if index == 0:
            header = list([a.value for a in row])
            continue
        obj = {}
        for col_index, cell in enumerate(row):
            value = cell.value
            obj[header[col_index]] = value
        values.append(obj)
        
    return values

In [56]:
rotineira2022 = get_sheet_values('Inspecoes_v2', 0)

In [6]:
prefix_blacklist = ["1) ","2) ","3) ","4) ","5) ","6) ","7) ","8) ","9) ","10) ","11) ","12) ","13) ","14) ","15) ",
                    "1. ","2. ","3. ","4. ","5. ","6. ","7. ","8. ","9. ","10. ","11. ","12. ","13. ","14. ","15. ",
                    "1.","2.","3.","4.","5.","6.","7.","8.","9.","10.","11.","12.","13.","14.","15.",
                    "1 – ","2 – ","3 – ","4 – ","5 – ","6 – ","7 – ","8 – ","9 – ","10 – ","11 – ","12 – ","13 – ","14 – ","15 – ",
                    "1 - ","2 - ","3 - ","4 - ","5 - ","6 - ","7 - ","8 - ","9 - ","10 - ","11 - ","12 - ", "13 - ","14 - ","15 - ",
                    "10 – ", "9  – ", "1- ",  "2 – ","4 –", "5– ", "6  - ", "3  – ", "8  – ", "5  – ", "4  – ", "6  – ", "-1. ",
                   "• ", "•"]

In [7]:
def separate_photos(value):
    value=tuple(value.split(' '))
    
    return value

In [8]:
def parse_terapias(terapias):
    terapias_list = ([
        b.strip(' .,;') for b in terapias.split('\n') 
        if b.strip(' .,;') 
        and b != "Por ora não há recomendações de terapia." 
        and b != "-"
        and b != "- Indicações de Terapia"]
        if terapias else [])
    ret_terapias = []
#     print(terapias_list)

    
    for terapia in terapias_list:
#         print(terapia)

        if terapia[0:2] in prefix_blacklist:
            terapia = terapia[2:]
        elif terapia[0:3] in prefix_blacklist:
            terapia = terapia[3:]            
        elif terapia[0:4] in prefix_blacklist:
            terapia = terapia[4:]
        elif terapia[0:5] in prefix_blacklist:
            terapia = terapia[5:]

        if terapia == "Implantação de sinalização horizontall":
            terapia = "Implantação de sinalização horizontal"
            
        if terapia == "- Indicações de Terapia":
            continue
#         print(terapia)
        

        
        terapia=terapia.replace('f','F').replace(').', ')').replace('.', ';').replace(' (', '; (').replace('.(', '; (').replace(';;', ';').replace(';(', '; (').replace('  ', ' ')
#         print(terapia)
#         if 'e' in terapia.split('; ')[1]:
#             print(terapia.split('; ')[1].strip('.,;)(').replace(',', '').replace(' e', '').replace(' 1', ' 01').replace(' 2', ' 02').replace(' 3', ' 03').replace(' 4', ' 04').replace(' 5', ' 05').replace(' 6', ' 06').replace(' 7', ' 07').replace(' 8', ' 08').replace(' 9', ' 09').replace('010', '10').replace('011', '11').replace('012', '12').replace('013', '13').replace('014', '14').replace('015', '15').replace('016', '16').replace('017', '17').replace('018', '18').replace('019', '19'))
#         terapia = (terapia.split('; ')[0].strip(' .,;'), separate_photos(terapia.split('; ')[1].strip(' .,;)(').replace(',', '').replace(' e', '').replace(' 1', ' 01').replace(' 2', ' 02').replace(' 3', ' 03').replace(' 4', ' 04').replace(' 5', ' 05').replace(' 6', ' 06').replace(' 7', ' 07').replace(' 8', ' 08').replace(' 9', ' 09').replace('010', '10').replace('011', '11').replace('012', '12').replace('013', '13').replace('014', '14').replace('015', '15').replace('016', '16').replace('017', '17').replace('018', '18').replace('019', '19').replace('020', '20').replace('021', '21').replace('022', '22').replace('Fotos', '').replace('Foto', '').replace('023', '23').replace('024', '24').replace('025', '25').replace('026', '26').replace('  ', ' ').strip()))
        terapia = (terapia.split('; ')[0].strip(' .,;'), separate_photos(terapia.split('; ')[1].strip(' .,;)(').replace(',', '').replace(' e', '').replace(' 01', ' 1').replace(' 02', ' 2').replace(' 03', ' 3').replace(' 04', ' 4').replace(' 05', ' 5').replace(' 06', ' 6').replace(' 07', ' 7').replace(' 08', ' 8').replace(' 09', ' 9').replace('010', '10').replace('011', '11').replace('012', '12').replace('013', '13').replace('014', '14').replace('015', '15').replace('016', '16').replace('017', '17').replace('018', '18').replace('019', '19').replace('020', '20').replace('021', '21').replace('022', '22').replace('Fotos', '').replace('Foto', '').replace('023', '23').replace('024', '24').replace('025', '25').replace('026', '26').replace('  ', ' ').strip()))
#         terapia[1] = separate_photos(terapia[1])
        ret_terapias.append(terapia)

        
        
#     print(terapias)
#     print(ret_terapias)
#     print('\n')
#     print('\n')
    return ret_terapias

In [9]:
all_terapias = []

In [61]:
for a in tqdm(rotineira2022):
    all_terapias += parse_terapias(a['Terapia 1'])

  0%|          | 0/185 [00:00<?, ?it/s]

In [10]:
def get_connected_reference(company, resource, key, value, default=None):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    try:
        return next(a['value'] for a in select_options if a['name'].lower() == value.lower())
    except StopIteration:
        return default

In [11]:
def get_form_reference(form, key, value):
    field = next(a for a in form.form_fields['fields'] if a['apiName'] == key)
    try:
        return next(a['value'] for a in field['selectOptions']['options'] if a['name'].lower() == value.lower())
    except StopIteration:
        print("DEURUIMM")
        print(form, key, value)
        return ""

In [12]:
def get_job_number(company):

    instance_type = "job"
    key_name = "{}_name_format".format(instance_type)
    # Get datetime and serial arrays
    data = get_autonumber_array(company, instance_type)
    # Get company prefix
    if "company_prefix" in company.metadata:
        data["prefixo"] = company.metadata["company_prefix"]
    else:
        data["prefixo"] = "[{}]".format(company.name)
    # Make number
    try:
        if key_name in company.metadata:
            number = company.metadata[key_name].format(**data)
        else:
            raise Exception("Variáveis de nome inválidas!")
    except Exception as e:
        print(e)
        # Fallback
        # UHIT-job-2018.0001
        number = "{prefixo}-{nome}-{anoCompleto}.{serialAno}".format(**data)

    return number

In [13]:
def get_reporting_number(company, occurrence_kind):

    key_name = "RP_name_format"
    number_format = ""

    if key_name in company.metadata:
        try:
            number_format = company.metadata[key_name][
                occurrence_kind
            ]
        except Exception:
            if "default" in company.metadata[key_name]:
                number_format = company.metadata[key_name][
                    "default"
                ]
            else:
                raise serializers.ValidationError(
                    "Variáveis de nome inválidas!"
                )
    else:
        raise serializers.ValidationError("Variáveis de nome inválidas!")

    instance_type = number_format["type"]

    # Get datetime and serial arrays
    data = get_autonumber_array(company.uuid, instance_type)

    # Get company prefix
    if "company_prefix" in company.metadata:
        data["prefixo"] = company.metadata["company_prefix"]
    else:
        data["prefixo"] = "[{}]".format(company.name)

    # Make number
    try:
        number = number_format["format"].format(**data)
    except Exception as e:
        print(e)
        # Fallback
        # UHIT-RG-2018.0001
        number = "{prefixo}-{nome}-{anoCompleto}.{serialAno}".format(**data)

    return number

In [14]:
def get_direction_name(company, value):
    choices=company.custom_options['reporting']['fields']['direction']['select_options']['options']
#     items={a['name']: a['short'] for a in choices}
    items={a['value']: a['short'] for a in choices}
    
    return items[value].replace('/','')

In [15]:
def get_direction_value(company, name):
    choices=company.custom_options['reporting']['fields']['direction']['select_options']['options']
#     items={a['name']: a['short'] for a in choices}
    items={a['short'].replace('/',''): a['value'] for a in choices}
    
    return items[name.upper()]

In [16]:
executado = ServiceOrderActionStatus.objects.get(companies=company, name="Executado")
identificado = ServiceOrderActionStatus.objects.get(companies=company, name="Identificado")
firm = Firm.objects.get(company=company, name="Não atribuído")
form_inspecao_rotineira = OccurrenceType.objects.get(company=company, name="Inspeção Rotineira")

In [64]:
objects = []
terapia_objects = []
jobs = []
photos=[]
photos_general=[]
counter_inside=0

for a in tqdm(rotineira2022):
    
#     km = float(a['km'].replace("+", "."))
    road_name = a['Rodovia'].replace(' ', '-').replace('/','-')
    road_name_photo = a['Rodovia'].replace(' ', '').replace('-','').replace('/','-')
#     print(road_name_photo)
    
    try:
        oae = Reporting.objects.get(company=company, form_data__artesp_service=a['Item de serviço ARTESP'], occurrence_type__occurrence_kind="2")
    except Exception as e:
        print(f"OAE não encontrada - Item da ARTESP {a['Item de serviço ARTESP']}")
        continue
        
    inspection_2022 = oae.children.filter(executed_at__year=2022, occurrence_type__name__icontains="rotineira")
    if inspection_2022.exists():
        print(f'Inspeção já existe - {inspection_2022.first().number}')
#         inspection_2022.first().delete()
        continue

    inspection_uuid = uuid4()   
    
    photos_uuid=defaultdict(list)
    
    terapias = parse_terapias(a['Terapia 1'])
    extensions=['.jpeg', '.jpg', '.png']
    
    for terapia in terapias:
        
        for photo_name in terapia[1]:
            counter_inside+=1
            for i, item in enumerate(extensions):
        
                photo=ReportingFile(upload='sisoae_2022_vpt_v2/' + road_name_photo + '/' + road_name_photo + '-km' + a['km'] + '-' + get_direction_name(company, oae.direction) + '/' + road_name_photo + '-km' + a['km'] + '-' + get_direction_name(company, oae.direction) + '/' + 'Foto' + photo_name + item,
                                uuid=uuid4(), description='Imagem da Terapia', reporting_id=inspection_uuid, created_by=arteriscs)
                photo_general=ReportingFile(upload='sisoae_2022_vpt_v2/' + road_name_photo + '/' + road_name_photo + '-km' + a['km'] + '-' + get_direction_name(company, oae.direction) + '/' + road_name_photo + '-km' + a['km'] + '-' + get_direction_name(company, oae.direction) + '/' + 'Foto' + photo_name + item,
                                uuid=uuid4(), description='Imagem da Terapia',  kind='antes', reporting_id=inspection_uuid, created_by=arteriscs)
                try:
                    photo.upload.size
                    photo_general.upload.size
                except Exception as e:
                    if i == 2:
                        print(f'{road_name_photo}/{road_name_photo}-km{a["km"]}-{get_direction_name(company, oae.direction)}/{road_name_photo}-km{a["km"]}-{get_direction_name(company, oae.direction)}/Foto{photo_name}')
                    continue
                else:
                    photos_uuid[terapia[0]].append(str(photo.uuid))
                    photos.append(photo)
                    photos_general.append(photo_general)
                    break
#     continue
     
#     print(photos_uuid)
    terapias = [
        {"description": terapia, "occurrence_type":"f7858877-f5a5-4416-ba1a-39e5f320b30d", "treatment_images": photo_uuid}
        for terapia, photo_uuid in photos_uuid.items() if terapia is not None and terapia != []
    ]
#     print(terapias)
    
    try:
        structural_classification = get_form_reference(form_inspecao_rotineira, 'structuralClassification', a['Estrutural'].strip()) if not "-" in a['Estrutural'] else ""
        functional_classification = get_form_reference(form_inspecao_rotineira, 'functionalClassification', a['Funcional'].strip()) if not "-" in a['Funcional'] else ""
        wear_classification = get_form_reference(form_inspecao_rotineira, 'wearClassification', a['Durabilidade'].strip()) if not "-" in a['Durabilidade'] else ""
    except Exception as e:
        print(f'Erro na função "get_form_reference" - {e}')
        structural_classification = ""
        functional_classification = ""
        wear_classification = ""
        
    if isinstance(a['Encontrado em'], str):
        a['Encontrado em'] = datetime.strptime(a['Encontrado em'], "%d/%m/%Y")
        
    try:
        obj = {
            "uuid": inspection_uuid,
            "number": get_reporting_number(company, form_inspecao_rotineira.occurrence_kind),
            "km": oae.km,
            "end_km": oae.end_km,
            'direction': oae.direction,
            'lane': oae.lane,
            'occurrence_type': form_inspecao_rotineira,
            'status': executado,
            'executed_at': a['Encontrado em'],
            'found_at': a['Encontrado em'],
            'created_at': a['Encontrado em'],
            'road_name': oae.road_name,
            'parent_id': oae.uuid,
            'form_data': {
                'repairs': a['Reparos'].replace('\n', '') if a['Reparos'] is not None else None,
                'reform': a['Reformas'].replace('\n', '') if a['Reformas'] is not None else None,
                'intervention': a['Reforços'].replace('\n', '') if a['Reforços'] is not None else None,
                'visual_structure_board': a['Característica visual do tabuleiro'].replace('\n', '')if a['Característica visual do tabuleiro'] is not None else None,
                'visual_structure_joints': a['Característica visual das juntas de dilatação'].replace('\n', '') if a['Característica visual das juntas de dilatação'] is not None else None, 
                'visual_structure_supports': a['Característica visual dos aparelhos de apoio'].replace('\n', '') if a['Característica visual dos aparelhos de apoio'] is not None else None,
                'visual_structure_walls': a['Característica visual dos pilares'].replace('\n', '') if a['Característica visual dos pilares'] is not None else None,
                'visual_structure_connections': a['Característica visual dos encontros'].replace('\n', '') if a['Característica visual dos encontros'] is not None else None,
                'paving_conditions': a['Pavimento'].replace('\n', '') if a['Pavimento'] is not None else None,
                'coasting': a['Acostamento'].replace('\n', '') if a['Acostamento'] is not None else None,
                'drains': a['Drenagem'].replace('\n', '') if a['Drenagem'] is not None else None,
                'guard_rail': a['Guarda-Corpos'].replace('\n', '') if a['Guarda-Corpos'] is not None else None,
                'defenses': a['Defensas rígidas'].replace('\n', '') if a['Defensas rígidas'] is not None else None,
                'slope': a['Taludes'].replace('\n', '') if a['Taludes'] is not None else None,
                'lightning': a['Iluminação'].replace('\n', '') if a['Iluminação'] is not None else None,
                'signaling': a['Sinalização'].replace('\n', '') if a['Sinalização'] is not None else None,
                'templates': a['Gabaritos'].replace('\n', '') if a['Gabaritos'] is not None else None,
                'dolfins': a['Dolfins'].replace('\n', '') if a['Dolfins'] is not None else None,
                'additional_info': a['Informações complementares'].replace('\n', '') if a['Informações complementares'] is not None else None,
                "construction": oae.form_data['construction'] if 'construction' in oae.form_data and oae.form_data['construction'] is not None else None,
                'therapy': terapias if terapias and terapias != [] else None,
                'structural_classification': structural_classification,
                'functional_classification': functional_classification, 
                'wear_classification': wear_classification,
                "artesp_service": a["Item de serviço ARTESP"],
            },
            'created_by': arteriscs,
            'point': oae.point,
            'road': oae.road,
            'company': oae.company,
            'firm': firm
        }
        try:
            if not obj['form_data']['therapy']:
                del obj['form_data']['therapy']
            inspection = Reporting(**obj)
            objects.append(inspection)
        except Exception as e:
            print(f'Erro de criação de terapia do item {a["Item de serviço ARTESP"]} - {e}')
            continue
    except AttributeError as e:
        print(f'Erro geral - {e}')
        continue
                  
    continue
        
    job_title = "Fase 2 - {}".format(oae.form_data['artesp_service'])
    job_color = "#7030A0"
    job_exists = Job.objects.filter(title=job_title, company=company).exists()
    
    if job_exists:
        job_title = "Fase 3 - {}".format(oae.form_data['artesp_service'])
        job_color = "#00CCFF"
        
    job = Job(
        title=job_title,
        description=job_color,
        number=get_job_number(company),
        start_date=a['Encontrado em'],
        end_date=a['Encontrado em'] + timedelta(days=365),
        inspection=inspection,
        parent_inventory=oae,
        created_by=arteriscs,
        firm=firm,
        worker=arteriscs,
        company=company
    )
    jobs.append(job)
    
    for terapia in terapias:
        form_terapia = OccurrenceType.objects.get(company=company, uuid=terapia['occurrence_type'])
        obj = {
            "number": get_reporting_number(company, form_terapia.occurrence_kind),
            "km": oae.km,
            "end_km": oae.end_km,
            'direction': oae.direction,
            'lane': oae.lane,
            'occurrence_type': form_terapia,
            'status': identificado,
            'found_at': a['Encontrado em'],
            'created_at': a['Encontrado em'],
            'road_name': oae.road_name,
            'parent_id': oae.uuid,
            'form_data': {
                'description': terapia['description']
            },
            'created_by': arteriscs,
            'point': oae.point,
            'road': oae.road,
            'company': oae.company,
            'firm': firm,
            'job': job
        }
        try:
            terapia_obj = Reporting(**obj)
            terapia_objects.append(terapia_obj)
        except Exception:
            print('terapia error')

  0%|          | 0/185 [00:00<?, ?it/s]

In [65]:
len(list(photos_general))

580

In [66]:
result1 = bulk_create_with_history(objects, Reporting, default_user=arteriscs)
result2 = bulk_create_with_history(photos, ReportingFile, default_user=arteriscs)
result3 = bulk_create_with_history(photos_general, ReportingFile, default_user=arteriscs)

/home/ubuntu/hidros-backend/venv/lib/python3.6/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2022-07-01 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/ubuntu/hidros-backend/venv/lib/python3.6/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2022-07-01 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/ubuntu/hidros-backend/venv/lib/python3.6/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2022-07-19 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/ubuntu/hidros-backend/venv/lib/python3.6/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2022-07-19 00:00:00) while time zone support is active.


In [56]:
reportings=Reporting.objects.filter(company=company, occurrence_type=form_inspecao_rotineira, created_at__gte='2022-12-10').order_by('number')
reportings.count()

/home/ubuntu/hidros-backend/venv/lib/python3.6/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.created_at received a naive datetime (2022-12-10 00:00:00) while time zone support is active.
  RuntimeWarning)


411

In [57]:
for a in tqdm(reportings):
    if a.form_data['therapy'] is None:
        print(a.uuid)
        del a.form_data['therapy']
        a.save()

  0%|          | 0/411 [00:00<?, ?it/s]

In [5]:
file_name='Lista de Fotos - SISOAE 2022'

with open(file_name + '.txt', 'r+') as fo:
    photo_list=json.load(fo)

In [25]:
photos=[]

for key, values in tqdm(photo_list.items()):
    
    try:
        reporting=Reporting.objects.get(company=company, occurrence_type=form_inspecao_rotineira, created_at__gte='2022-12-10', created_by=arteriscs, road_name=key.split('*')[0].replace('-', '/').replace(' ', '-'), km=key.split('*')[1].replace('+', '.'), direction=get_direction_value(company, key.split('*')[2]))
        for value in values:
            photo=ReportingFile(upload='sisoae_2022_vpt/' + key.split('*')[0] + '/' + key.split('*')[1] + ' ' + key.split('*')[2] + '/' + value,
                            uuid=uuid4(), description='Imagem da Inspeção', kind='antes', reporting_id=reporting.uuid, created_by=arteriscs)
            
            try:
                photo.upload.size
                photos.append(photo)
            except Exception as e:
                print(key, value)
    except Exception as e:
        print(key)
        pass

  0%|          | 0/411 [00:00<?, ?it/s]

/home/ubuntu/hidros-backend/venv/lib/python3.6/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.created_at received a naive datetime (2021-11-26 00:00:00) while time zone support is active.
  RuntimeWarning)


SP 255*178+080*T
SP 255*179+020*T


In [26]:
# photo_result = bulk_create_with_history(photos, ReportingFile, default_user=arteriscs)

In [68]:
from RoadLabsAPI.settings import credentials
import boto3



# session = boto3.client(
#     "s3",
#     aws_access_key_id=credentials.AWS_ACCESS_KEY_ID,
#     aws_secret_access_key=credentials.AWS_SECRET_ACCESS_KEY,
# )

s3 = boto3.resource('s3')

In [70]:
pdf_list={}

In [71]:
my_bucket = s3.Bucket('spotway-roadlabs-assets')

for my_bucket_object in my_bucket.objects.filter(Prefix="media/private/sisoae_2022_vpt_v2/pdf"):
    if '.pdf' in my_bucket_object.key:
        pdf_list[my_bucket_object.key.split('/')[-1].split('-')[0]] = my_bucket_object.key.split('/')[-1]

In [72]:
pdf_list

{'SP255': 'SP255-km356+200-NS.pdf',
 'SP257': 'SP257-km010+200-NS.pdf',
 'SP304': 'SP304-km295+850-T.pdf',
 'SP318': 'SP318-km277+180-NS.pdf',
 'SP328': 'SP328-km310+500-NS.pdf',
 'SP330': 'SP330-km318+480-T.pdf',
 'SP334': 'SP334-km403+200-T.pdf',
 'SP345': 'SP345-km039+050-T.pdf',
 'SPA020': 'SPA020-345-km002+000-NS.pdf',
 'SPA074': 'SPA074-255-km003+900-T.pdf',
 'SPA214': 'SPA214-255-km000+200-NS.pdf',
 'SPA307': 'SPA307-330-km002+600-T.pdf',
 'SPA318': 'SPA318-330-km001+100-T.pdf',
 'SPA348': 'SPA348-334-km002+700-NS.pdf'}

In [18]:
oaes=Reporting.objects.filter(company=company, occurrence_type__occurrence_kind="2")
oaes.count()

416

In [19]:
inspection_2022.road_name.replace('-','').replace('/','-')

NameError: name 'inspection_2022' is not defined

In [20]:
pdf_upload=[]

for oae in tqdm(oaes):
    
    try:
        inspection_2022 = oae.children.filter(executed_at__year=2022, created_at__year=2023, occurrence_type__name__icontains="rotineira").first()
        if inspection_2022:
            km="{:07.3f}".format(inspection_2022.km).replace(".", "+")
            pdf_file=ReportingFile(upload='sisoae_2022_vpt_v2/pdf/' + inspection_2022.road_name.replace('-','').replace('/','-') + '-km'+ km + "-" + get_direction_name(company, oae.direction) + ".pdf",
                                uuid=uuid4(), description='PDF', reporting_id=inspection_2022.uuid, created_by=arteriscs)
            pdf_file.upload.size
            pdf_upload.append(pdf_file)    
    except Exception as e:
        if inspection_2022:
            print(f"{inspection_2022.road_name.replace('-','').replace('/','-')}-km{km}-{get_direction_name(company, oae.direction)}")
            print(oae.form_data['artesp_service'])
            pass

  0%|          | 0/416 [00:00<?, ?it/s]

In [21]:
len(pdf_upload)

185

In [22]:
result = bulk_create_with_history(pdf_upload, ReportingFile, default_user=arteriscs)